In [1]:
#####
##### Generative n-gram some experiments
##### strartin from Yoav Goldberg model
#####

from collections import *
import os

##### Training by the orginal model  
def train_char_lm(data, order=6):
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [2]:
##### I use italian Zibaldone by Giacomo Leopardi as basin dataset
!wget -O 'data/Zibaldone.zip' https://www.liberliber.it/mediateca/libri/l/leopardi/pensieri_di_varia_filosofia_etc/txt/pensieri.zip --no-check-certificate

--2018-06-25 12:42:39--  https://www.liberliber.it/mediateca/libri/l/leopardi/pensieri_di_varia_filosofia_etc/txt/pensieri.zip
Resolving www.liberliber.it... 93.186.244.67
Connecting to www.liberliber.it|93.186.244.67|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 2388125 (2.3M) [application/zip]
Saving to: 'data/Zibaldone.zip'

data/Zibaldone.zip  100%[===================>]   2.28M  7.54MB/s    in 0.3s    

2018-06-25 12:42:40 (7.54 MB/s) - 'data/Zibaldone.zip' saved [2388125/2388125]



In [3]:
import zipfile
with zipfile.ZipFile('data/Zibaldone.zip','r') as zip_ref:
    zip_ref.extractall('/Users/marcofosci/Desktop/0 - Artificial Intelligence/0 - NLP/Github/p2/data/')
filename = 'data/pensieri.txt'
with open(filename, encoding="latin-1") as f:
    data = f.read()

In [4]:
from random import random

##### The generative letter model
def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

In [5]:
##### ...and then the generative text model with some modification
def generate_text(lm, initWord, order, nletters=500):
    history = initWord
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

In [6]:
train = train_char_lm(data, order=6)

In [7]:
print("The probability of the next character for 6-gram 'giocar': ") 
print(train['giocar'])
print("The probability of the next character 6-gram 'mangia': ")
print(train['mangia'])

The probability of the next character for 6-gram 'giocar': 
[('e', 1.0)]
The probability of the next character 6-gram 'mangia': 
[('r', 0.5), ('t', 0.2916666666666667), ('s', 0.041666666666666664), ('n', 0.125), ('b', 0.020833333333333332), (',', 0.020833333333333332)]


In [8]:
##### A simple function to take casual string from the dataset 
def dataStart(data, lenght):
    x = int(random()*len(data))
    out = data[x:x+lenght]
    print(out)
    return out, x

In [9]:
start, ind = dataStart(data, 6)
print()
print(data[ind+6:ind+800])

, e no

n solamente, come oggi, all'intelletto; (sebbene anche al solo intelletto può parlare la lingua italiana, se vuole) pieghevole, robusta, o delicata secondo l'occorrenza; piena di sève, di sangue e di colorito ec. ec. Delle quali proprietà qualche avanzo se ne può notare nella Sévigné, e nel Bossuet e in altri scrittori di quel tempo. Talmente che s'ella fosse rimasta quale ho detto, non sarebbe mai stata universale, con che vengo a dir tutto. E s'ella prima della sua mortifera riforma, avesse avuto tanto numero di cultori quanto n'ebbe l'italiana, che l'avessero condotta secondo il suo carattere primitivo, e d'allora, alla perfezione, come fu condotta la nostra, sarebbe anche più evidente questo ch'io dico [690]della prima e originale natura della lingua francese, la quale ben si con


In [10]:
generate_text(train, start, 6, 800)

"n quelle cose che dice mai, e non può una certo. \n(7. Dec. 1822.)\n  \n  Tio spazio. Tanto più indistruggono grado di un indicativa dell'uomo, anzi se tutta una certo il sistema, più scemare le moderna, massime accade rispettive des Romano in corpo agguagliardi ciascuno indubitarne matematica, e circostantemente consisterebbe mai non si può onninamente.) Cum his hands; that he culte pubblica. Ecco rinovato all'andamento separar di particolarmente similissimi propongono la formata che, sono come particella perfezione, e un libro 1. 13. Aprile. 1821.)\n  \n  Ciascuno è che scritture, stupenda. Questo è vera, vasello. Ora dovuti, come si dubito una perfetto, cioè se nel buono attivo di chi si guardasse di più formidabile, until I shall own a great and Mucius Scaevola persona che il gran rilievo p"

In [11]:
##### Now we clean the original dataset 
##### First we delete the header that contains non-literary data
print(data[:2700])
data = data[2510:]

Giacomo Leopardi
Pensieri di varia filosofia e di bella letteratura

Questo e-book è stato realizzato anche grazie al sostegno di:
E-text
Editoria, Web design, Multimedia
http://www.e-text.it/

QUESTO E-BOOK:

TITOLO: Pensieri di varia filosofia e di bella letteratura
AUTORE: Leopardi, Giacomo
TRADUTTORE:
CURATORE:
NOTE: l'opera è conosciuta anche come "Zibaldone".
I passaggi in greco che contiene l'opera sono stati tutti realizzati con il font Athenian, ad eccezione della lettera "teta" presente nella tabella di Symbol.
Per realizzare il carattere "jod" presente nei paragrafi, §1127 e §1276, è stato utilizzato il font Lucida Sans Unicode (sottoinsieme "simboli alfabetici"); an-cora il Sans Unicode è stato scelto per esprimere il simbolo digamma coniato da Leopardi (§ 2197; § 4291) e le vocali lunghe (§ 2365). Per quanto concerne i ra-pidi passaggi del testo in ebraico, il font d'uso è l'Hebrew.
Nella versione solo testo del libro (.TXT), le parole in greco ed ebraico sono state sostit

In [12]:
##### In this experiment we are not interested in the graphic expression, the carriage and the spacing. 
##### We just want to see how much the generated text can be coherent.
##### Then we use just lower letter and remove some vocabolary symbols that it is not useful 
##### We use a simple function that could be useful also in future

import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@]'r"\n")
BAD_SYMBOLS_RE = re.compile('[^a-z ,.;:\']')

def text_prepare(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text

In [13]:
##### Before and after text preparation
print(data[:500])
print('____________________________')
data = text_prepare(data)
print(data[:500])

GIACOMO LEOPARDI
  
ZIBALDONE
Pensieri di varia filosofia e di bella letteratura
  
  
1]Palazzo bello. Cane di notte dal casolare, al passar del viandante.
  
  Era la luna nel cortile, un lato
  Tutto ne illuminava, e discendea
  Sopra il contiguo lato obliquo un raggio...
  Nella (dalla) maestra via s'udiva il carro
  Del passegger, che stritolando i sassi,
  Mandava un suon, cui precedea da lungi
  Il tintinnìo de' mobili sonagli.
  
  Onde Aviano raccontando una favoletta dice che una donna
____________________________
giacomo leopardi  zibaldonepensieri di varia filosofia e di bella letteratura    palazzo bello. cane di notte dal casolare, al passar del viandante.    era la luna nel cortile, un lato  tutto ne illuminava, e discendea  sopra il contiguo lato obliquo un raggio...  nella dalla maestra via s'udiva il carro  del passegger, che stritolando i sassi,  mandava un suon, cui precedea da lungi  il tintinno de' mobili sonagli.    onde aviano raccontando una favoletta dice che 

In [14]:
train = train_char_lm(data, order=10)

In [15]:
start, ind = dataStart(data, 10)
generate_text(train, start, 10, 1000)

mposti der


"iva non da un participii in us e ne' supini fu quale noi la vediamo, pessimamente in essa si citano. . luglio. .     alla p.. octobre,  cirey. . tome . lettres du prince de canino, n.. roma . dic. .     quello che fa nel plurale, perduto anche le capitali, si limita a queste, com' naturale per questa pu facilmente riconoscimento, altro soggetto di rimettere eziandio quello de' bardi britanni, p.. e soprattutto durando la novit delle metafore troppo vediamo, rimonteremo a quell'opera: non  una cosa in qualunque cosa, queste per la composizioni della universalmente dal cavalca, e gli scrittori d'immaginare, perch l'intelletto non si troverete ne' lessici greci sono stati effetti.  nell'iliade oggid massimamente le stesse cose che si vede n che cosa  per conseguenza elle non affatto stazionarii da traiano, la religione ec. ma con alcuna qualit positiva, quanto all'utilit di un medesimo argomenti storici, e quelli che da antichissimi, tende e produce or dunque non siano dominate da princi

In [16]:
train = train_char_lm(data, order=20)

In [17]:
start, ind = dataStart(data, 20)

 all'indecente, e fo


In [18]:
print(data[ind+20:ind+1000])

rse di necessit e contro voglia, l'ho portato gi troppo innanzi. dir brevemente. di quelle parti umane che taluno non conosce, o in quel tempo in cui nessuno le conosce, non solo non ne ha veruna idea di bello o di brutto, e volendola formare, verisimilissimamente s'inganna, ma volendo congetturare le loro propriet, forme e proporzioni universali, non indovina, se non forse a caso. e il fanciullo distingue gi il bello e il brutto fra gli uomini, e ancora non conosce intieramente la bellezza non solo, ma neppure la forma umana, e quello che ne conosce non gli d veruna idea sufficiente, n delle propriet n delle proporzioni e convenienze di quello che non conosce. e v. in questo proposito p.. marg. . luglio .     alla p.. marg.  e divenir maturo, pratico ec. p.e. in uno stile, con una sola lettura, cio con pochissimo esercizio ec. la qual facilit di assuefazione, segno ed effetto del talento io la notava in me anche nelle minuzie, come nell'assuefarmi ai diversi metodi


In [19]:
generate_text(train, start, 20, 1000)

"rse di necessit e contro voglia, l'ho portato gi troppo innanzi. dir brevemente. di quelle parti umane che taluno non conosce, o in quel tempo in cui essi son giunti a formarsi un gusto e un tatto fino e squisito in materie letterarie e in ogni altra cosa che ci appartenga non  in sua mano l'alterarle, l'indebolirle, lo stravolgerle, l'estinguerle la nostra medesima ragione non  tutta quanta in balia della fortuna pu nessuno assicurarsi o vantarsi di non aver mai a perder l'uso della ragione, anzi nell'incremento. v. p.. capoverso ult.    al capoverso superiore. e perci appunto  necessario che le opere classiche antiche tradotte perdano tutto o quasi tutto alla natura, e pochissimo o nulla differiva dalla parlata e volgare. ma altrettanta e forse maggiore uniformit non senza ragione di verit, che bisogna soddisfare ai desiderii de' fanciulli per non trovargli morti dietro alle porte. . sett. .     fermezza di carattere  di due sorti, che nascono da principii affatto contrarii, l'una d

In [20]:
generate_text(train, start, 20, 1000)

"rse di necessit e contro voglia, l'ho portato gi troppo innanzi. dir brevemente. di quelle parti umane che taluno non conosce, o in quel tempo in cui essi son giunti a formarsi un gusto e un tatto fino e squisito in materie letterarie e in ogni altra cosa. e queste qualit divengono in loro caratteristiche, di modo che l'amor proprio  cagione d'infelicit; giacch un desiderio non soddisfatto di piacere, e il desiderio della felicit. l'ubbriachezza e tutto ci che le si assomiglia o le appartiene ec.  piacevole per sua natura, e volerla applicare alle cose moderne ec. . nov. .                 ,          ,   . e che il quinto libro                      .                 .    cunctis idem si pulchrum, et egregium foret,  nulla esset anceps hominibus contentio.  at nunc simile nil, nil idem mortalibus:  nisi verba forsan inter istos concinunt,  at re tamen, factisque convenit nihil.    e cicerone de fin. bon. et mal. c.. verso il fine: sed quot homines, tot sententiae: falli igitur possumus.

In [38]:
##### We have used a dataset with 5.6 mln of charachter
len(data)

5657721

In [21]:
##### Now we change the dataset with a more modern I Promessi Sposi by Alessanddro Manzoni 
!wget -O 'data/PromessiSposi.txt' http://rosada.yolasite.com/resources/i_promessi%20sposi.txt

--2018-06-25 12:44:44--  http://rosada.yolasite.com/resources/i_promessi%20sposi.txt
Resolving rosada.yolasite.com... 104.16.159.215, 104.16.160.215, 104.16.163.215, ...
Connecting to rosada.yolasite.com|104.16.159.215|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: 'data/PromessiSposi.txt'

data/PromessiSposi.     [ <=>                ]   1.25M  --.-KB/s    in 0.1s    

2018-06-25 12:44:44 (11.1 MB/s) - 'data/PromessiSposi.txt' saved [1312204]



In [22]:
filename = 'data/PromessiSposi.txt'
with open(filename, encoding="latin-1") as f:
    data_2 = f.read()

In [23]:
##### I remove some not literary data
data = data_2[1790:]
##### ...and also other bad characters
print(data[:500])
print('_______________________________')
data = text_prepare(data)
print(data[:500])


I promessi sposi
di Alessandro Manzoni



INTRODUZIONE

"L'Historia si può veramente deffinire una guerra illustre contro il Tempo, perché togliendoli di mano gl'anni suoi prigionieri, anzi già fatti cadaueri, li richiama in vita, li passa in rassegna, e li schiera di nuovo in battaglia. Ma gl'illustri Campioni che in tal Arringo fanno messe di Palme e d'Allori, rapiscono solo che le sole spoglie più sfarzose e brillanti, imbalsamando co' loro inchiostri le Imprese de Prencipi e Potentati, e qu
_______________________________
i promessi sposidi alessandro manzoniintroduzionel'historia si pu veramente deffinire una guerra illustre contro il tempo, perch togliendoli di mano gl'anni suoi prigionieri, anzi gi fatti cadaueri, li richiama in vita, li passa in rassegna, e li schiera di nuovo in battaglia. ma gl'illustri campioni che in tal arringo fanno messe di palme e d'allori, rapiscono solo che le sole spoglie pi sfarzose e brillanti, imbalsamando co' loro inchiostri le imprese de prenci

In [24]:
train = train_char_lm(data, order=10)

In [25]:
start, ind = dataStart(data, 10)

ine si smo


In [26]:
generate_text(train, start, 10, 1000)

"nterebbe allora in poi ho paura che gli si diceva esser la loro strada. era costui  tanto il manico dell'aspo che facevan coraggio tuttavia, con significativo, un restare al di sotto, e, un po' d'opinione avesse intenzion d'aiutarlo: cap benissimo.  un galantuomo,  risponde a tutto il paese rischiarato dalla grande smania che quel silenzio, che, a quel che ora si direbbe che la pi parte per quel maledetto il signor curato: propriamente, non derogando, eccetera. ce n' della roba buona. e poi andate a letto presto, presto, presto, presto, con due o tre altri salotti oscuri, arrivarono inaspettato, agnese trov ogni cosa in confusione di poca importanza; perch il signor duca di feria ai  di dicembre, che fu il giorno stabilito, la processione: i due compagnia di cantambanchi, quando, cessato ogni pericolo, vi corse incontri di morte che ci si badasse pi che lei voleva far confessarmi, supponiamo, tu n'avessi ammazzati la giustizia da s, in qualche offerta. si ricorda che s' fissato per la

In [27]:
train = train_char_lm(data, order=20)

In [28]:
start, ind = dataStart(data, 20)

a aggrottate: teneva


In [31]:
generate_text(train, start, 20, 1000)

" in mano una supplica, e pareva che dicesse: ah la c' cascata la brava. quella vista, risvegliando pi vivi nell'animo suo tutti gli antichi sentimenti, le restitu anche un po' di quel poco antico coraggio: e gi stava cercando una risposta qualunque, diversa da quella che le era stata dettata; quando, alzato lo sguardo alla faccia del padrone, che s'era rannuvolata: l'uno e l'altro avrebber voluto riprender quella di prima; ma non era possibile. gli altri convitati pensavano, ognun da s, al modo di sopire il piccolo scandolo, e di fare una diversione; ma, pensando, tacevano, e, in quel silenzio, lo scandolo era pi manifesto. ognuno scansava d'incontrar gli occhi degli altri; ognuno sentiva che tutti eran occupati del pensiero che tutti volevan dissimulare. la gioia, per quel giorno, e per qualche altro ancora. ma che essi medesimi poi, chiacchierando con la gente del paese, e senza voler mostrar di saperne pi di loro, quando si veniva a quel punto oscuro della fuga de' nostri tre pover

In [32]:
print(data[ind:ind+1000])

a aggrottate: teneva in mano una supplica, e pareva che dicesse: vedremo. di qua una matrona, terrore delle sue cameriere; di l un abate, terrore de' suoi monaci: tutta gente in somma che aveva fatto terrore, e lo spirava ancora dalle tele. alla presenza di tali memorie, don rodrigo tanto pi s'arrovellava, si vergognava, non poteva darsi pace, che un frate avesse osato venirgli addosso, con la prosopopea di nathan. formava un disegno di vendetta, l'abbandonava, pensava come soddisfare insieme alla passione, e a ci che chiamava onore; e talvolta vedete un poco sentendosi fischiare ancora agli orecchi quell'esordio di profezia, si sentiva venir, come si dice, i bordoni, e stava quasi per deporre il pensiero delle due soddisfazioni. finalmente, per far qualche cosa, chiam un servitore, e gli ordin che lo scusasse con la compagnia, dicendo ch'era trattenuto da un affare urgente. quando quello torn a riferire che que' signori eran partiti, lasciando i loro rispetti:  e il conte attilio  dom